<a href="https://colab.research.google.com/github/DeepaManjunath/Assignment5-New/blob/master/Assignment5Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe
from keras.preprocessing.image import ImageDataGenerator
import keras


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Using TensorFlow backend.


In [0]:
tf.enable_eager_execution()

In [0]:

BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 24 #@param {type:"integer"}

In [0]:

def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:

class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
    self.drop = tf.keras.layers.Dropout(0.03555)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.drop(self.conv(inputs))))

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h


In [10]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct


In [13]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, (EPOCHS-4)], [0.050, LEARNING_RATE, 0.050])[0] 
                                  
                                 
global_step = tf.train.get_or_create_global_step()
print(global_step)
print(lr_schedule)
lr_schedule1 = lambda t: np.interp([t], [(EPOCHS-3), (EPOCHS-3), (EPOCHS)], [0.025, 0.025, 0.0])[0]

print(batches_per_epoch)
global_step1 = tf.train.get_or_create_global_step()

lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE

lr_func1=lambda: lr_schedule1(global_step1/batches_per_epoch)/BATCH_SIZE

opt = tf.train.MomentumOptimizer(lr_func,momentum=MOMENTUM,use_nesterov=False)


data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)


<tf.Variable 'global_step:0' shape=() dtype=int64, numpy=0>
<function <lambda> at 0x7f24bf2a6400>
98


In [14]:
x_train.shape

(50000, 40, 40, 3)

In [0]:
import numpy as np

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [0]:
datagen=ImageDataGenerator(      
                                                                rescale=1./255,width_shift_range=0.1,
	                                                              height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	                                                              horizontal_flip=True, fill_mode="nearest", 
                                                                rotation_range=25,preprocessing_function=get_random_eraser(v_l=0, v_h=1)   
)
	                                                             
datagen.fit(x_train)


In [17]:

t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range((EPOCHS)):
  
    train_loss = test_loss = train_acc = test_acc = 0.0
    train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

    tf.keras.backend.set_learning_phase(1)
    for (x, y) in tqdm(train_set):
      with tf.GradientTape() as tape:
        loss, correct = model(x,y)

      var = model.trainable_variables
      grads = tape.gradient(loss, var)
      for g, v in zip(grads, var):
           g += v * WEIGHT_DECAY * BATCH_SIZE
      if (epoch> (EPOCHS-3)):
         opt.apply_gradients(zip(grads, var), global_step=global_step1)
      else:
         opt.apply_gradients(zip(grads, var), global_step=global_step)
 

      train_loss += loss.numpy()
      train_acc += correct.numpy()

    tf.keras.backend.set_learning_phase(0)
    for (x, y) in test_set:
        loss, correct = model(x,y)
        test_loss += loss.numpy()
        test_acc += correct.numpy()
    print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)
    
    train_loss = test_loss = train_acc = test_acc = 0.0
    train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

    tf.keras.backend.set_learning_phase(1)
    for (x, y) in tqdm(train_set):
        with tf.GradientTape() as tape:
          loss, correct = model(x,y)

        var = model.trainable_variables
        grads = tape.gradient(loss, var)
        for g, v in zip(grads, var):
           g += v * WEIGHT_DECAY * BATCH_SIZE
        if epoch> (EPOCHS-3):
          opt.apply_gradients(zip(grads, var), global_step=global_step1)
        else:
          opt.apply_gradients(zip(grads, var), global_step=global_step1)
 

        train_loss += loss.numpy()
        train_acc += correct.numpy()

    tf.keras.backend.set_learning_phase(0)
    for (x, y) in test_set:
      loss, correct = model(x,y)
      test_loss += loss.numpy()
      test_acc += correct.numpy()
    print('epoch:', epoch+1, 'lr:', lr_schedule1(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.12000000000000001 train loss: 1.4368409729003906 train acc: 0.47 val loss: 1.6647215942382811 val acc: 0.463 time: 40.20719242095947



epoch: 1 lr: 0.025 train loss: 0.8794431921386718 train acc: 0.6885 val loss: 1.580740447998047 val acc: 0.5532 time: 65.78850984573364



epoch: 2 lr: 0.19 train loss: 0.6892596032714844 train acc: 0.75986 val loss: 0.8675473388671875 val acc: 0.7122 time: 91.27263140678406



epoch: 2 lr: 0.025 train loss: 0.5978241390991211 train acc: 0.79494 val loss: 0.6520292953491211 val acc: 0.7796 time: 116.9180965423584



epoch: 3 lr: 0.26 train loss: 0.5308436114501953 train acc: 0.81624 val loss: 2.088266107177734 val acc: 0.5438 time: 142.3879759311676


...


epoch: 22 lr: 0.01666666666666667 train loss: 0.017543212127685545 train acc: 0.99576 val loss: 0.2925042694091797 val acc: 0.926 time: 1137.179123878479



epoch: 23 lr: 0.05 train loss: 0.01716275514125824 train acc: 0.99562 val loss: 0.28560011596679685 val acc: 0.9294 time: 1162.7258341312408



epoch: 23 lr: 0.008333333333333335 train loss: 0.01642109769821167 train acc: 0.99624 val loss: 0.30895719833374025 val acc: 0.9236 time: 1188.1436700820923



epoch: 24 lr: 0.05 train loss: 0.016358706588745116 train acc: 0.99578 val loss: 0.288268359375 val acc: 0.9278 time: 1213.6217486858368



epoch: 24 lr: 0.0 train loss: 0.015484585025310516 train acc: 0.99634 val loss: 0.28646054611206057 val acc: 0.9278 time: 1239.0829331874847
